# Seminar 8. Generators, Iterators

Example from warm up test. Don't create collections in the function signature. They will be created once, with function. This can lead to undesirable behavior and leaks.

In [ ]:
from typing import Optional, List

# wrong
def foo(a=1, b=[], c='abc'):
    a += 1
    b.append(2)
    c += 'd'


# ok
def foo(a=1, b=None, c='abc'):
    a += 1
    b = b or []
    b.append(2)
    c += 'd'


# ok but with annotations
def foo(a: int = 1, b: Optional[List] = None, c: str = 'abc'):
    a += 1
    b = b or []
    b.append(2)
    c += 'd'

Another example from warm up test. It shows that decorator with params is a feature, made with additional wrapper. It even can be applied twice.

In [ ]:
import functools

def log_call(param1, param2):
    def temp1(param3):
        def temp2(func):
            @functools.wraps(func)
            def wrapper(*args, **kwargs):
                print(f'{func.__name__} called with args={args}, kwargs={kwargs} and params {param1}, {param2} and {param3}')
                result = func(*args, **kwargs)
                print(f'{func.__name__} result={result}')
                return result
            return wrapper
        return temp2
    return temp1


"""
since Python 3.9 this syntax supported

@log_call('param1', 'param2')('param3')
def my_func():
    print('abc')
"""

# before Python 3.9 additional expression needed
temp_deco = log_call('param1', 'param2')
@temp_deco('param3')
def my_func():
    print('abc')


"""
What will happend?

# Step1 - temp1 with param1 and param2 in closure will be used
@temp1('param3')
def my_func():
    print('abc')

# Step2 - temp2 with param3 and temp1's scope in closure will be used
@temp2
def my_func():
    print('abc')


# Step3 - decoration will happen
my_func = temp2(my_func)


# Step4 - my_func replaced with wrapper. wrapper has all params in its closure.
my_func = wrapper
"""

my_func()

my_func called with args=(), kwargs={} and params param1, param2 and param3
abc
my_func result=None


### Tricky unpacking example

In [ ]:
for x, y in [(1, 2), (3, 4)]:
    print(x, y)

for name, (x, y) in {'a': (1, 2), 'b': (3, 4)}.items():
    print(name, ':', x, y)

1 2
3 4
a : 1 2
b : 3 4


## Iterable and iterators

Glossary: https://docs.python.org/3/glossary.html#term-iterable

In [ ]:
class MyRange:
    """
    Analog for `range` with single param.

    MyRange's instance is an iterator.
    """
    def __init__(self, end):
        self.current = 0
        self.end = end

    def __iter__(self):
        return self

    def __next__(self):
        if self.current == self.end:
            raise StopIteration
        current = self.current
        self.current += 1
        return current


r = MyRange(10)
print(r)
iterator = iter(r)
print(iterator)
step1 = next(iterator)
print(step1)
step2 = next(iterator)
print(step2)
# ...
# step10 = next(iterator)
# step11 = next(iterator)  # not allowed

0
1


Standard Python construnctions support iterator protocol (`__iter__`, `__next__`, `StopIteration` exception)

In [ ]:
for i in MyRange(10):
    print(i)

min(MyRange(10)), max(MyRange(10))

0
1
2
3
4
5
6
7
8
9


(0, 9)

That's how `for` loop works for random iterable object

In [ ]:
r = iter(range(10))
while True:
    try:
        i = next(r)
        print(i)
    except StopIteration:
        break

0
1
2
3
4
5
6
7
8
9


`iter` returns `self` according to iterator protocol. so it won't create new iterator.

In [ ]:
mr_iterable = MyRange(10)
mr1_iterator = iter(mr_iterable)
print(next(mr1_iterator))
print(next(mr1_iterator))
mr2_iterator = iter(mr_iterable)
print(next(mr2_iterator))
print(next(mr2_iterator))

0
1
2
3


However, default `range`'s instance is an iterable. We can create new iterators from this iterable.

In [ ]:
r_iterable = range(10)
r1_iterator = iter(r_iterable)
print(next(r1_iterator))
print(next(r1_iterator))
r2_iterator = iter(r_iterable)
print(next(r2_iterator))
print(next(r2_iterator))

0
1
0
1


But we can't create new iterators from iterators.

In [ ]:
r_iterable = range(10)
print(r_iterable, type(r_iterable))

r1_iterator = iter(r_iterable)  # iterable's __iter__ creates new iterator
print(r1_iterator, type(r1_iterator))
print(next(r1_iterator))
print(next(r1_iterator))

r2_iterator = iter(r1_iterator)  # iterator's __iter__ returns self
print(r2_iterator, type(r2_iterator))
print(next(r2_iterator))
print(next(r2_iterator))

range(0, 10) <class 'range'>
<range_iterator object at 0x7f54a0639e40> <class 'range_iterator'>
0
1
<range_iterator object at 0x7f54a0639e40> <class 'range_iterator'>
2
3


Let's create MyRange as an iterable

In [ ]:
class MyRangeIterator:
    """
    MyRangeIterator's instance is an iterator.
    """
    def __init__(self, end):
        self.current = 0
        self.end = end

    def __iter__(self):
        return self

    def __next__(self):
        if self.current == self.end:
            raise StopIteration
        current = self.current
        self.current += 1
        return current


class MyRange:
    """
    MyRange's instance is an iterable.
    """
    def __init__(self, end):
        self.end = end

    def __iter__(self):
        return MyRangeIterator(self.end)


mr_iterable = MyRange(10)
mr1_iterator = iter(mr_iterable)
print(next(mr1_iterator))
print(next(mr1_iterator))
mr2_iterator = iter(mr_iterable)
print(next(mr2_iterator))
print(next(mr2_iterator))

0
1
0
1


## Generator

Glossary: https://docs.python.org/3/glossary.html#index-19

Generator is a way to create iterator (not iterable) using functions and `yield` statement.

In [ ]:
def myrange(end):
    current = 0
    while current < end:
        yield current
        current += 1


mr_iterable = myrange(10)  # returns self
mr1_iterator = iter(mr_iterable)  # that won't create new iterator, mr_iterable is already iterator
print(next(mr1_iterator))
print(next(mr1_iterator))
mr2_iterator = iter(mr_iterable)  # that won't create new iterator, mr_iterable is already iterator
print(next(mr2_iterator))
print(next(mr2_iterator))


print('-' * 10)


for i in myrange(10):  # iterator protocol works
    print(i)


print('-' * 10)


mr = myrange(3)
print(mr)
print(next(mr))
print(next(mr))
print(next(mr))
print(next(mr))  # StopIteration also raises when function finished

0
1
2
3
----------
0
1
2
3
4
5
6
7
8
9
----------
<generator object myrange at 0x7f54a063abd0>
0
1
2


StopIteration: ignored

Regular functions don't support iterator protocol

In [ ]:
def myrange(end):
    current = 0
    while current < end:
        return current
        current += 1

mr = myrange(5)
print(next(mr))

TypeError: ignored

Generator statement is also the way to create an iterator.

It looks like list comprehension with parenthesis but Python doesn't create all objects in a memory, just like with generator.

In [ ]:
generator_statement = (i**2 for i in range(1689147169487164897163248713264918320))
print(generator_statement)
print(next(generator_statement))
print(next(generator_statement))
print(next(generator_statement))

<generator object <genexpr> at 0x7f54a063ab50>
0
1
4
